In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib

# Strategy parameters
name = "btc"
time = "2023"
timefra = "1h"
bollinger_period = 30
rsi_period = 14
upper_band_multiplier = 2
lower_band_multiplier = 2
quantity = 1

def run_backtest(file_path, take_profit_percent, stop_loss_percent):
    # Load and prepare the data
    data = pd.read_csv(file_path, header=None)
    data.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 
                    'CloseTime', 'QuoteAssetVolume', 'NumberOfTrades', 
                    'TakerBuyBaseAssetVolume', 'TakerBuyQuoteAssetVolume', 'Ignore']
    data['Timestamp'] = pd.to_datetime(data['Timestamp'], unit='ms')
    data.set_index('Timestamp', inplace=True)
    data['Close'] = data['Close'].astype(float)

    # Calculate indicators
    data['RSI'] = talib.RSI(data['Close'], timeperiod=rsi_period)
    data['UpperBB'], data['MiddleBB'], data['LowerBB'] = talib.BBANDS(
        data['Close'], timeperiod=bollinger_period, nbdevup=upper_band_multiplier, 
        nbdevdn=lower_band_multiplier)

    for period in [12, 144, 169, 576, 676]:
        data[f'EMA{period}'] = talib.EMA(data['Close'], timeperiod=period)

    short_vol = data['Volume'].rolling(window=1).mean()
    long_vol = data['Volume'].rolling(window=14).mean()
    data['VolumeOsc'] = short_vol - long_vol

    # Initialize trading flags and variables
    in_position = False
    buy_signal = False
    buy_signal2 = False
    buy_signal3 = False
    entry_price = 0
    peak_price = 0
    trade_records = []
    transaction_cost_percent = 0.1 / 100
    last_buy_time = None

    # Backtesting loop
    for current_time, current_row in data.iterrows():
        if in_position:
            # Update peak price
            peak_price = max(peak_price, current_row['Close'])

            # Check for sell conditions
            if (current_row['Close'] >= entry_price * (1 + take_profit_percent)) or \
               (current_row['Close'] <= peak_price * (1 - stop_loss_percent)) or \
               (current_row['Close'] < current_row['LowerBB'] and current_row['RSI']<20) or \
               ((current_row['Close'] < current_row['EMA676']) and (current_row['VolumeOsc'] > 144) and ((current_time - entry_time).total_seconds() > 10000)):
                # Calculate profit with transaction cost for entry and exit
                sell_price = current_row['Close']
                entry_cost = entry_price * quantity * transaction_cost_percent
                exit_cost = sell_price * quantity * transaction_cost_percent
                profit = (sell_price - entry_price) * quantity - entry_cost - exit_cost
                in_position = False
                holding_period = (current_time - entry_time).total_seconds() / 60  # Convert to minutes
                trade_records.append({
                    'HoldingPeriod': holding_period,
                    'EntryPrice': entry_price,
                    'ExitPrice': sell_price,
                    'Profit': profit,
                    'ExitTime': current_time
                })

        else:
            # First part of the new buy condition
            if (current_row['EMA144']>current_row['EMA576']) and (current_row['EMA169']>current_row['EMA576'])and current_row['EMA12']>current_row['EMA144'] and current_row['Close']<current_row['EMA144']:
                buy_signal=True
            
            elif buy_signal== True and current_row['Close']>current_row['EMA144']:
                buy_signal= False
                
            
            elif buy_signal==True and current_row['Close']<current_row['EMA169']:
                buy_signal= False
                buy_signal2=True

            elif buy_signal2==True and current_row['Close']<current_row['EMA576']:
                buy_signal2=False
                buy_signal3=True
                
            elif buy_signal3==True and current_row['Close']>current_row['EMA169'] and current_row['EMA12']>current_row['EMA169'] and current_row['RSI']>68 and (current_row['EMA144']>current_row['EMA576']) and (current_row['EMA169']>current_row['EMA576']):
                    in_position = True
                    entry_price = current_row['Close']
                    entry_cost = entry_price * quantity * transaction_cost_percent
                    peak_price = entry_price
                    entry_time = current_time
                    buy_signal3 = False

                    
            elif buy_signal2==True and current_row['Close']>current_row['EMA169'] and current_row['EMA12']>current_row['EMA169'] and current_row['RSI']>68 and (current_row['EMA144']>current_row['EMA576']) and (current_row['EMA169']>current_row['EMA576']):
                    in_position = True
                    entry_price = current_row['Close']
                    entry_cost = entry_price * quantity * transaction_cost_percent
                    peak_price = entry_price
                    entry_time = current_time
                    buy_signal2 = False  # Reset buy signal

    return pd.DataFrame(trade_records)

def aggregate_optimal_parameters(file_paths, start_tp=0.02, end_tp=0.15, start_sl=0, end_sl=0.1, step=0.01):
    best_take_profit = start_tp
    best_stop_loss = start_sl
    highest_aggregate_profit = -float('inf')

    for take_profit in np.arange(start_tp, end_tp + step, step):
        for stop_loss in np.arange(start_sl, end_sl + step, step):
            aggregate_profit = 0

            # Run backtest for each file and sum the profits
            for file_path in file_paths:
                trade_records_df = run_backtest(file_path, take_profit, stop_loss)
                aggregate_profit += trade_records_df['Profit'].sum() if not trade_records_df.empty else 0

            # Check if this combination gives the highest profit
            if aggregate_profit > highest_aggregate_profit:
                highest_aggregate_profit = aggregate_profit
                best_take_profit = take_profit
                best_stop_loss = stop_loss

    return best_take_profit, best_stop_loss, highest_aggregate_profit

# Paths for the CSV files
file_paths = {''
}

# Running the optimization for each file
print("Optimizing across all files...")
best_take_profit, best_stop_loss, highest_aggregate_profit = aggregate_optimal_parameters(file_paths)
print("\nOptimal parameters across all files:")
print("Best Take Profit Percent:", best_take_profit)
print("Best Stop Loss Percent:", best_stop_loss)
print("Highest Aggregate Cumulative Profit:", highest_aggregate_profit)


Optimizing across all files...

Optimal parameters across all files:
Best Take Profit Percent: 0.09
Best Stop Loss Percent: 0.06
Highest Aggregate Cumulative Profit: 7765.8577999999925
